In [3]:
import io
import os
import matplotlib.pyplot as plt
import numpy as np 
import pandas as pd 

import boto3
import sagemaker
from sagemaker import get_execution_role

%matplotlib inline

In [4]:
# sagemaker session, role
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

# S3 bucket name
bucket = sagemaker_session.default_bucket()



In [5]:
s3 = boto3.resource('s3')
b = s3.Bucket('sagemaker-mlai-harvesting')
!mkdir data
b.download_file( 'data/MLAI_ParsedDataSet.tsv', 'data/data.tsv')

# !head data/data.tsv

In [28]:
txn = pd.read_csv('data/data.tsv',sep='\t')
txn.head()

,Act,Activity,Log_Time,CustID,CustName,MarketID,CustCountry,CustState,GroupID,ProfID,...,Search_Term,Num_of_Results,DbName,AN,Retrieval_Format,Retrieval_Mode,Checkout_Result,Checkout_Type,Violation_Type,Bad_Actor
0,121,Search Click,2019-05-01 19:00:00.000,s8989762,COLUMBIA COLLEGE,academic,UNITED STATES,ILLINOIS,main,ehost,...,new jobs new inequalities,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,115,Retrievals,2019-05-01 19:00:00.000,s8983751,UNIV OF LA VERNE,academic,UNITED STATES,CALIFORNIA,main,ehost,...,NaN,NaN,aph,96191495,Abstract,View,NaN,NaN,NaN,0
2,115,Retrievals,2019-05-01 19:00:01.000,s8969716,SHENANDOAH UNIVERSITY,academic,UNITED STATES,VIRGINIA,main,ehost,...,NaN,NaN,e000xna,171851,Abstract,View,NaN,NaN,NaN,0
3,111,Session Start,2019-05-01 19:00:01.000,s8987536,FLORIDA A & M UNIV,academic,UNITED STATES,FLORIDA,main,wsapi,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,121,Search Click,2019-05-01 19:00:01.000,s8987536,FLORIDA A & M UNIV,academic,UNITED STATES,FLORIDA,main,wsapi,...,mass incarceration,17533.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [190]:
txn.columns

Index(['Act', 'Activity', 'Log_Time', 'CustID', 'CustName', 'MarketID',
       'CustCountry', 'CustState', 'GroupID', 'ProfID', 'Int_ID', 'Client_IP',
       'UserID', 'CookieGUID', 'SessionNo', 'UserAgent', 'Login_Method',
       'Personalization', 'Search_From', 'EP_Db_Count', 'Search_Term',
       'Num_of_Results', 'DbName', 'AN', 'Retrieval_Format', 'Retrieval_Mode',
       'Checkout_Result', 'Checkout_Type', 'Violation_Type', 'Bad_Actor'],
      dtype='object')

In [202]:
txns = np.sort(txn['Act'].unique())
txni= pd.DataFrame( txns )
txni.columns = ['txn']
txni['txni']= txni.index
# txni.set_index('txn')
txni['col'] = ['txn_' + str(y) for y in range(len(txni))]
txni


,txn,txni,col
0,111,0,txn_0
1,112,1,txn_1
2,114,2,txn_2
3,115,3,txn_3
4,116,4,txn_4
5,117,5,txn_5
6,118,6,txn_6
7,119,7,txn_7
8,121,8,txn_8
9,123,9,txn_9


In [221]:
def flatten_txns( txn_log, txn_types ):
    txn_narrow = txn_log[['SessionNo', 'Act','Bad_Actor']]
    txn_narrowi = txn_narrow.merge( txn_types, left_on='Act', right_on='txn'  )
    txn_narrowi  = txn_narrowi[['SessionNo', 'Bad_Actor',  'col' ]]
    txn_pivot = pd.pivot_table(txn_narrowi, index=['SessionNo','Bad_Actor'], columns = ['col'],aggfunc=[np.size]).fillna(0)
    txn_pivot.columns = txn_pivot.columns.droplevel(0)
    txn_flat = txn_pivot.rename_axis(None, axis=1).reset_index()
    return txn_flat

In [224]:
flatten_txns( txn, txni ).head(10)

,SessionNo,Bad_Actor,txn_0,txn_1,txn_10,txn_11,txn_12,txn_13,txn_14,txn_15,...,txn_30,txn_31,txn_32,txn_33,txn_4,txn_5,txn_6,txn_7,txn_8,txn_9
0,-2147317281,0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,0.0
1,-2147002735,0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,6.0,0.0
2,-2146953899,0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,48.0,0.0
3,-2146926264,0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0
4,-2146915841,0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0
5,-2146723372,0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,3.0,0.0
6,-2146089473,0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,3.0,3.0,0.0,0.0,0.0,0.0
7,-2145083328,0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,3.0,3.0,0.0,0.0,6.0,6.0
8,-2145021172,0,0.0,0.0,0.0,0.0,27.0,24.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,-2144988764,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [226]:
flat = flatten_txns( txn, txni )

In [230]:
def func_split( flat ):
    flat_x = flat.drop('Bad_Actor', axis=1)
    flat_y = flat['Bad_Actor']
    return flat_x, flat_y

def train_split( flat ):
    flat_bad = flat[flat['Bad_Actor']==1]
    flat_good = flat[flat['Bad_Actor']==0]
    train = 
    


In [238]:
flat_y

0        0
1        0
2        0
3        0
4        0
5        0
6        0
7        0
8        0
9        0
10       0
11       0
12       0
13       0
14       0
15       0
16       0
17       0
18       0
19       0
20       0
21       0
22       0
23       0
24       0
25       0
26       0
27       0
28       0
29       0
        ..
15220    0
15221    0
15222    0
15223    0
15224    0
15225    0
15226    0
15227    0
15228    0
15229    0
15230    0
15231    0
15232    0
15233    0
15234    0
15235    0
15236    0
15237    0
15238    0
15239    0
15240    0
15241    0
15242    0
15243    0
15244    0
15245    0
15246    0
15247    0
15248    0
15249    0
Name: Bad_Actor, Length: 15250, dtype: int64